### **Assignment 3: FastText**

# **Description:**
Requirements:
- Pick many documents → [Yelp Dataset](https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset/data) "yelp_academic_dataset_tip.json" (Use only Text
column)
- Preprocessing to text
## **Step-1:**
- Train FastText using genism on preprocessed texts → generate/save your own FastText
model
Test your model:
- Print 10 similar words & 10 opposite words for a given/input words
## **Step-2:**
Test pretrained model:

- Download pretrained FastText model https://dl.fbaipublicfiles.com/fasttext/vectors-
crawl/cc.en.300.bin.gz

- Print 10 similar words & 10 opposite words for a given/input words
## **Step-3:**
- Update the pretrained model using your own data (Yelp_tip)
- Print 10 similar words & 10 opposite words for a given/input words
In a Pdf Report:
➢ Compare the output of the three steps for the same (given/input words)
➢ You must have a conclusion/comments

### **Authors**:

| Name                           | ID          |
|--------------------------------|-------------|
| Ro'aa Fathi                    | 20210140    |
| Selsabeel Asim                 | 20210714    |
| Abdelrahman Mohamed            | 20210518    |
| Nourhan Hossam                 | 20221185    |



Task distribution:
2 people working on steps 2 & 3,
2 people working on implementing the bonus (refer to the second part of the fasttext lecture for details!)

[Report](https://docs.google.com/document/d/1ASp9_i2pQ9Itgwm0prQx5wIlSFBH6kP9Sp9k_bP-kyU/edit?tab=t.0)

## **Imports & Loading Dataset**

In [ ]:
#run this, and will restart the session, then contniue
!pip uninstall -y numpy gensim scipy
!pip install numpy==1.23.5 gensim==4.3.2 scipy==1.10.1

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 37.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.61.0 requires numpy<2.2,>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
scikit-image 0.25.2 requires numpy

In [ ]:
import pandas as pd
import numpy
import gensim
from gensim.models import FastText
from nltk.tokenize import word_tokenize
import re
import nltk
import zipfile
from nltk.stem import WordNetLemmatizer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from typing import List, Tuple

In [ ]:
!pip install kagglehub

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

upload ur kaggle.json in runtime

In [ ]:
!kaggle datasets download -d yelp-dataset/yelp-dataset

Dataset URL: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset
License(s): other


In [ ]:
%%capture
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
with zipfile.ZipFile("yelp-dataset.zip", 'r') as zip_ref:
    zip_ref.extract("yelp_academic_dataset_tip.json", path="/content/yelp_data")

In [ ]:
file_path = '/content/yelp_data/yelp_academic_dataset_tip.json'
data = pd.read_json(file_path, lines=True)
texts = data['text'].dropna().sample(n=5000, random_state=42).tolist()
print(data.head())

                  user_id             business_id  \
0  AGNUgVwnZUey3gcPCJ76iw  3uLgwr0qeCNMjKenHJwPGQ   
1  NBN4MgHP9D3cw--SnauTkA  QoezRbYQncpRqyrLH6Iqjg   
2  -copOvldyKh1qr-vzkDEvw  MYoRNLb5chwjQe3c_k37Gg   
3  FjMQVZjSqY8syIO-53KFKw  hV-bABTK-glh5wj31ps_Jw   
4  ld0AperBXk1h6UbqmM80zw  _uN0OudeJ3Zl_tf6nxg5ww   

                                                text                date  \
0                     Avengers time with the ladies. 2012-05-18 02:17:21   
1  They have lots of good deserts and tasty cuban... 2013-02-05 18:35:10   
2             It's open even when you think it isn't 2013-08-18 00:56:08   
3                          Very decent fried chicken 2017-06-27 23:05:38   
4             Appetizers.. platter special for lunch 2012-10-06 19:43:09   

   compliment_count  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  


## **Preprocessing Steps**

In [ ]:
def tokenize(text):
    return word_tokenize(text)

In [ ]:
def lowercase(tokens):
    return [word.lower() for word in tokens]

In [ ]:
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

In [ ]:
def remove_special_chars(tokens):
    return [word for word in tokens if word.isalpha()]

In [ ]:
def remove_extra_whitespace(text):
    return " ".join(text.split())

In [ ]:
def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in tokens]

In [ ]:
def preprocess_text(text):
    text =  remove_extra_whitespace(text)
    tokens = tokenize(text)
    tokens = lowercase(tokens)
    tokens = remove_special_chars(tokens)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize(tokens)
    return tokens

In [ ]:
preprocessed_texts = [preprocess_text(text) for text in texts]

In [ ]:
print(preprocessed_texts)

[['herb', 'roasted', 'salmon', 'great'], ['audition'], ['one', 'fave', 'spot', 'tampa', 'bay', 'double', 'yum'], ['store', 'well', 'organized', 'impressed'], ['want', 'traditional', 'old', 'diner', 'experience', 'belly', 'marble', 'counter', 'order', 'slice', 'homemade', 'pie'], ['seeing', 'pogo'], ['literally', 'worst', 'service', 'ever', 'received', 'restaurant', 'bus', 'boy', 'actually', 'took', 'food', 'finished', 'threw', 'away', 'proceeded', 'force', 'u', 'door', 'simply', 'unacceptable', 'behavior', 'suggest', 'eating'], ['ask', 'heat', 'spicy', 'enough'], ['adhere', 'hour', 'nobody', 'sight', 'saturday'], ['super', 'helpful', 'friendly', 'sale', 'folk', 'go', 'extra', 'mile'], ['hand', 'best', 'beer', 'selection', 'city'], ['food', 'friend'], ['hipster', 'vibe', 'overpriced', 'good', 'food'], ['crest', 'colgate', 'sensodyne', 'toothpaste', 'party', 'aquafresh', 'toothpaste', 'offered'], ['open', 'tuesday'], ['sunny', 'breezy', 'patio', 'lunch'], ['eggplant', 'parm', 'best', 'sl

## **Step 1: Train a Custom FastText Model**

In [ ]:
model = FastText(sentences=preprocessed_texts, vector_size=100, window=5, min_count=5, epochs=10)
model.save('fasttext_yelp.model')

In [ ]:
word = input("Enter a word: ")
try:
    print(f"\nTop 10 similar words to given input '{word}':")
    print(model.wv.most_similar(word, topn=10))

    print(f"\nTop 10 opposite words to given input'{word}':")
    print(model.wv.most_similar(negative=[word], topn=10))
except KeyError:
    print(f"'{word}' not found in vocabulary.")


Enter a word: trial

Top 10 similar words to given input 'trial':
[('aerial', 0.8112183213233948), ('tutorial', 0.8018279671669006), ('triage', 0.798321545124054), ('tribal', 0.7779898047447205), ('triathlon', 0.7673912048339844), ('denial', 0.7586470246315002), ('serial', 0.7427411675453186), ('burial', 0.7271615862846375), ('memorial', 0.7233531475067139), ('trivial', 0.7173791527748108)]

Top 10 opposite words to given input'trial':
[('iowa', 0.3427709639072418), ('wa', 0.33593571186065674), ('eh', 0.2973765432834625), ('sj', 0.2882067561149597), ('cushy', 0.2639671862125397), ('pushy', 0.2604637145996094), ('is', 0.24769467115402222), ('ehh', 0.23121604323387146), ('nasty', 0.22884300351142883), ('meh', 0.22377149760723114)]


## **Step 2: Use the Pretrained FastText Model**

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gunzip -k cc.en.300.bin.gz

--2025-04-12 22:14:04--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.171.22.13, 3.171.22.33, 3.171.22.68, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.171.22.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G   231MB/s    in 18s     

2025-04-12 22:14:23 (233 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [ ]:
from gensim.models import FastText


model = FastText.load_fasttext_format("/content/cc.en.300.bin")

<ipython-input-20-bca4b15ef88b>:4: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format("/content/cc.en.300.bin")


In [ ]:
word = input("Enter a word: ")
try:
    print(f"\nTop 10 similar words to given input '{word}':")
    print(model.wv.most_similar(word, topn=10))

    print(f"\nTop 10 opposite words to given input'{word}':")
    print(model.wv.most_similar(negative=[word], topn=10))
except KeyError:
    print(f"'{word}' not found in vocabulary.")

Enter a word: trial

Top 10 similar words to given input 'trial':
[('trials', 0.7523017525672913), ('trial.The', 0.7495211958885193), ('Trial', 0.7256538271903992), ('trial.This', 0.7206138372421265), ('trial-', 0.6947095990180969), ('trial.', 0.6824939846992493), ('trial.If', 0.6721920371055603), ('non-trial', 0.6691030263900757), ('trial.In', 0.6564455628395081), ('trial.It', 0.6538047194480896)]

Top 10 opposite words to given input'trial':
[('0878', 0.2127452939748764), ('fluorite', 0.21049237251281738), ('deadfred.com', 0.20934297144412994), ('8539', 0.20917852222919464), ('.Glass', 0.20718641579151154), ('97500', 0.20696961879730225), ('16803', 0.20623549818992615), ('8873', 0.20242513716220856), ('8412', 0.20221561193466187), ('9593', 0.20187297463417053)]


## **Step 3: Update the pretraind Model with Yelp Data**


In [ ]:
from gensim.models import FastText, KeyedVectors


model = FastText.load_fasttext_format("/content/cc.en.300.bin")

<ipython-input-22-3f7dbb46a501>:4: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format("/content/cc.en.300.bin")


In [ ]:
model.build_vocab(preprocessed_texts, update=True)
model.train(preprocessed_texts, total_examples=len(preprocessed_texts), epochs=10)

(20287636, 99089600)

In [ ]:
word = input("Enter a word: ")
try:
    print(f"\nTop 10 similar words to given input '{word}':")
    print(model.wv.most_similar(word, topn=10))

    print(f"\nTop 10 opposite words to given input'{word}':")
    print(model.wv.most_similar(negative=[word], topn=10))
except KeyError:
    print(f"'{word}' not found in vocabulary.")

Enter a word: trial

Top 10 similar words to given input 'trial':
[('non-trial', 0.9698066115379333), ('trial-by-trial', 0.9663671851158142), ('trial-to-trial', 0.9657558798789978), ('re-trial', 0.9622019529342651), ('mid-trial', 0.9620057344436646), ('pre-trial', 0.960676372051239), ('thetrial', 0.9591758251190186), ('Post-trial', 0.9585362672805786), ('mock-trial', 0.9517011642456055), ('Trial', 0.9494549036026001)]

Top 10 opposite words to given input'trial':
[('2014X', 0.1419043391942978), ('NaÃ', 0.14078845083713531), ('Guy11', 0.13381485641002655), ('55See', 0.1281866729259491), ('John11', 0.12748223543167114), ('Brown5', 0.1271524578332901), ('2017Ray', 0.12490511685609818), ('20Tom', 0.12394361197948456), ('2011E', 0.1231757402420044), ('Περιγραφή', 0.12283968180418015)]


## **Bonus Task**

In [ ]:
model = FastText(sentences=preprocessed_texts, vector_size=100, window=5, min_count=5, epochs=10)

In [ ]:
class FastTextModel(nn.Module):
    def __init__(self, vocab, embedding_dim, n_gram=3, num_neg_samples=5):
        super(FastTextModel, self).__init__()
        self.vocab = list(vocab)
        self.word_to_idx = {word: idx for idx, word in enumerate(self.vocab)}
        self.idx_to_word = {idx: word for word, idx in self.word_to_idx.items()}
        self.vocab_size = len(self.vocab)
        self.embedding_dim = embedding_dim
        self.n_gram = n_gram
        self.num_neg_samples = num_neg_samples

        self.word_embeddings = nn.Embedding(self.vocab_size, embedding_dim)
        self.subword_embeddings = nn.Embedding(self.vocab_size, embedding_dim)
        nn.init.xavier_uniform_(self.word_embeddings.weight)
        nn.init.xavier_uniform_(self.subword_embeddings.weight)

    @staticmethod
    def generate_dataset(documents, window_size=2):
        dataset = []
        for doc in documents:
            for i, target in enumerate(doc):
                context = []
                for j in range(max(0, i - window_size), min(len(doc), i + window_size + 1)):
                    if i != j:
                        context.append(doc[j])
                dataset.append((target, context))
        return dataset

    def get_subword(self, word):
        word = f'<{word}>'
        subwords = set()
        min_n = min(3, len(word))
        max_n = min(self.n_gram, len(word))
        for n in range(min_n, max_n + 1):
            subwords.update([word[i:i+n] for i in range(len(word) - n + 1)])
        return [word] + list(subwords)

    def get_word_vector(self, word_idx, subword_indices):
        word_vec = self.word_embeddings(word_idx)
        if len(subword_indices) > 0:
            subword_vecs = self.subword_embeddings(subword_indices).mean(dim=0)
            return word_vec + subword_vecs
        return word_vec

    def forward(self, target_word, context_word, label):
        target_idx = torch.tensor([self.word_to_idx.get(target_word, 0)], dtype=torch.long)
        context_idx = torch.tensor([self.word_to_idx.get(context_word, 0)], dtype=torch.long)

        target_subwords = self.get_subword(target_word)
        context_subwords = self.get_subword(context_word)

        target_subword_indices = [self.word_to_idx.get(sw, 0) for sw in target_subwords if sw in self.word_to_idx]
        context_subword_indices = [self.word_to_idx.get(sw, 0) for sw in context_subwords if sw in self.word_to_idx]

        target_subword_indices = torch.tensor(target_subword_indices, dtype=torch.long) if target_subword_indices else torch.tensor([0], dtype=torch.long)
        context_subword_indices = torch.tensor(context_subword_indices, dtype=torch.long) if context_subword_indices else torch.tensor([0], dtype=torch.long)

        target_vec = self.get_word_vector(target_idx, target_subword_indices)
        context_vec = self.get_word_vector(context_idx, context_subword_indices)

        dot_product = torch.dot(target_vec.squeeze(), context_vec.squeeze())
        pred = torch.sigmoid(dot_product.unsqueeze(0))
        label_tensor = torch.tensor([label], dtype=torch.float32)

        loss = F.binary_cross_entropy(pred, label_tensor)
        return loss

    def train_model(self, documents, epochs=5, batch_size=64, lr=0.001):
        dataset = self.generate_dataset(documents)
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)

        for epoch in range(epochs):
            random.shuffle(dataset)
            total_loss = 0.0
            print(f"\nEpoch {epoch+1}/{epochs}")

            for batch_idx, i in enumerate(range(0, len(dataset), batch_size)):
                batch = dataset[i:i+batch_size]
                optimizer.zero_grad()
                batch_loss = 0.0

                for target, context in batch:
                    for ctx_word in context:
                        loss_pos = self.forward(target, ctx_word, 1)
                        batch_loss += loss_pos

                        for _ in range(self.num_neg_samples):
                            neg_word = random.choice(self.vocab)
                            while neg_word in context or neg_word == target:
                                neg_word = random.choice(self.vocab)
                            loss_neg = self.forward(target, neg_word, 0)
                            batch_loss += loss_neg

                avg_loss = batch_loss / (len(batch) * (1 + self.num_neg_samples))
                avg_loss.backward()
                optimizer.step()
                total_loss += avg_loss.item()

                print(f"  Batch {batch_idx+1}/{(len(dataset) - 1)//batch_size + 1}, Batch Loss: {avg_loss.item():.4f}")

            print(f"Epoch {epoch+1} completed. Total Loss: {total_loss:.4f}")

    def get_top_similar_words(self, word, top_n=10):
        if word not in self.word_to_idx:
            return []

        word_idx = torch.tensor([self.word_to_idx[word]], dtype=torch.long)
        subwords = self.get_subword(word)
        subword_indices = [self.word_to_idx.get(sw, 0) for sw in subwords if sw in self.word_to_idx]
        subword_indices = torch.tensor(subword_indices, dtype=torch.long) if subword_indices else torch.tensor([0], dtype=torch.long)

        with torch.no_grad():
            target_vec = self.get_word_vector(word_idx, subword_indices).unsqueeze(0)

            all_vecs = []
            all_words = []

            for other_word, other_idx in self.word_to_idx.items():
                if other_word == word:
                    continue
                other_idx_tensor = torch.tensor([other_idx], dtype=torch.long)
                other_subwords = self.get_subword(other_word)
                other_subword_indices = [self.word_to_idx.get(sw, 0) for sw in other_subwords if sw in self.word_to_idx]
                other_subword_indices = torch.tensor(other_subword_indices, dtype=torch.long) if other_subword_indices else torch.tensor([0], dtype=torch.long)
                other_vec = self.get_word_vector(other_idx_tensor, other_subword_indices)
                all_vecs.append(other_vec)
                all_words.append(other_word)

            all_vecs = torch.stack(all_vecs)
            cos_sim = F.cosine_similarity(target_vec, all_vecs, dim=1)
            top_scores, top_indices = torch.topk(cos_sim, min(top_n, len(all_words)))

            results = [(all_words[i], top_scores[idx].item()) for idx, i in enumerate(top_indices)]
            return results


In [ ]:
def test_model_with_word(model, test_word):
    if test_word not in model.word_to_idx:
        print(f"'{test_word}' not in vocabulary")
        return

    word_idx = torch.tensor([model.word_to_idx[test_word]])
    word_subwords = model.get_subword(test_word)
    subword_indices = [model.word_to_idx.get(sw, 0) for sw in word_subwords if sw in model.word_to_idx]
    subword_indices = torch.tensor(subword_indices) if subword_indices else torch.tensor([0])

    with torch.no_grad():
        word_vec = model.get_word_vector(word_idx, subword_indices)

    test_words = [w for w in model.vocab if w != test_word][:5]  # Compare with first 5 other words
    print(f"\nTesting '{test_word}' against:", test_words)

    for other_word in test_words:
        other_idx = torch.tensor([model.word_to_idx[other_word]])
        other_subwords = model.get_subword(other_word)
        other_subword_indices = [model.word_to_idx.get(sw, 0) for sw in other_subwords if sw in model.word_to_idx]
        other_subword_indices = torch.tensor(other_subword_indices) if other_subword_indices else torch.tensor([0])

        with torch.no_grad():
            other_vec = model.get_word_vector(other_idx, other_subword_indices)

        cos_sim = torch.dot(word_vec.squeeze(), other_vec.squeeze()) / (
            torch.norm(word_vec) * torch.norm(other_vec)
        )

        print(f"Similarity with '{other_word}': {cos_sim.item():.3f}")

In [ ]:
vocab = set(word for sentence in preprocessed_texts for word in sentence)
model = FastTextModel(vocab=vocab, embedding_dim=100)
model.train_model(preprocessed_texts[:300], epochs=3, batch_size=16)


Epoch 1/3
  Batch 1/133, Batch Loss: 2.1663
  Batch 2/133, Batch Loss: 2.4328
  Batch 3/133, Batch Loss: 2.1748
  Batch 4/133, Batch Loss: 2.3898
  Batch 5/133, Batch Loss: 2.2633
  Batch 6/133, Batch Loss: 2.3001
  Batch 7/133, Batch Loss: 2.4771
  Batch 8/133, Batch Loss: 2.3001
  Batch 9/133, Batch Loss: 2.2127
  Batch 10/133, Batch Loss: 2.2554
  Batch 11/133, Batch Loss: 1.8643
  Batch 12/133, Batch Loss: 2.2564
  Batch 13/133, Batch Loss: 1.9081
  Batch 14/133, Batch Loss: 2.1220
  Batch 15/133, Batch Loss: 2.2977
  Batch 16/133, Batch Loss: 2.3413
  Batch 17/133, Batch Loss: 2.0786
  Batch 18/133, Batch Loss: 2.2109
  Batch 19/133, Batch Loss: 2.0367
  Batch 20/133, Batch Loss: 2.2071
  Batch 21/133, Batch Loss: 2.1655
  Batch 22/133, Batch Loss: 2.2943
  Batch 23/133, Batch Loss: 2.0798
  Batch 24/133, Batch Loss: 1.9925
  Batch 25/133, Batch Loss: 2.4689
  Batch 26/133, Batch Loss: 2.2953
  Batch 27/133, Batch Loss: 2.2082
  Batch 28/133, Batch Loss: 2.2086
  Batch 29/133, Ba

In [ ]:
test_model_with_word(model, "happy")


Testing 'happy' against: ['crabfingers', 'catering', 'confused', 'tyler', 'friday']
Similarity with 'crabfingers': -0.583
Similarity with 'catering': -0.679
Similarity with 'confused': -0.676
Similarity with 'tyler': -0.677
Similarity with 'friday': 0.614


In [ ]:
test_model_with_word(model, "try")


Testing 'try' against: ['crabfingers', 'catering', 'confused', 'tyler', 'friday']
Similarity with 'crabfingers': -0.548
Similarity with 'catering': -0.746
Similarity with 'confused': -0.808
Similarity with 'tyler': -0.747
Similarity with 'friday': 0.659
